In [1]:
import matplotlib 
import ipywidgets as widgets
import pandas as pd
import io
import seaborn as sns
import openpyxl
from IPython.display import display, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual, GridspecLayout, Button, Layout,Label, HTML, HBox, Image, VBox, Box
from matplotlib_venn import venn2, venn3
from matplotlib import pyplot as plt
from traitlets import traitlets
from ipyevents import Event


In [2]:
#access file from Allen
mouse_means = pd.read_csv('https://idk-etl-prod-download-bucket.s3.amazonaws.com/aibs_mouse_ctx-hip_smart-seq/trimmed_means.csv')

# Get all gene and cell type: MOUSE
mouse_idx = list(mouse_means.columns.values)
mouse_listOfColumnNames = list(mouse_means['cluster_label'])


In [3]:
#rearrange mouse data
mouse_means2 = mouse_means.set_index('cluster_label').T.rename_axis('GeneID').rename_axis(None, axis = 1)

In [4]:
#allow for whole dataframe to print out
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [5]:
##Create a class of buttons which allows me to check which one was pressed
class LoadedButton(widgets.Button):
    """A button that can hold a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))


button_layout = widgets.Layout(border='solid 1px black', height='auto', width='auto')
def create_expanded_button(description, button_style):
    return LoadedButton(description=description, button_style=button_style, layout=button_layout)


In [6]:
##functions needed
def on_go_button_clicked(go_button):
    genes = [genes1.value, genes2.value, genes3.value]
    #cut list to two if two or fewer selected
    genes_b = list(filter(None, genes))
    if len(genes_b) < 2:
        warnbox.value == "Please pick at least two genes"
        #display ("Please pick at least two genes")
    else:
        venngraph(genes_b)

        
def on_reset_button_clicked(reset_button):  
    genes1.value == 'Type Here'
    genes2.value == 'Type Here'
    genes3.value == 'Type Here'
    genes = []

    
def on_value_change(change):
    #just be here
    print('')



In [7]:
## all the labels
spacer = widgets.Label('')
welcome = widgets.Label('Allen Transcriptomics Venn: Choose a threhold for expression and 2 or 3 genes')
genetitle = widgets.Label('Select Genes')

    
## The buttons/ widgets and what they do
genes1 =  widgets.Combobox(options= mouse_idx,description='Gene',placeholder='Type here',layout={'width': '80%'})
genes2 =  widgets.Combobox(options= mouse_idx,description='Gene',placeholder='Type here',layout={'width': '80%'})
genes3 =  widgets.Combobox(options= mouse_idx,description='Gene',placeholder='Type here',layout={'width': '80%'})


go_button = create_expanded_button('Create Venn', '')
go_button.observe(genes1, 'value')  
go_button.observe(genes2, 'value')  
go_button.observe(genes3, 'value')  
go_button.on_click(on_go_button_clicked)



style = {'description_width': 'initial'}
threshold = widgets.FloatText(
    value=0.5,
    description='Expression Threshold',
    disabled=False,
    style = style,
width = '80%')
threshold.observe(on_value_change, names='value')



In [8]:
%matplotlib inline
def venngraph(genes):
        venny1, venny2, venny3 = [],[],[]
        venny = mouse_means2.loc[genes]

        venny1 = venny.loc[genes[0]]
        venny1 = venny1[venny1 >= threshold.value].index.to_list()
        venny1 = set(venny1)

        venny2 = venny.loc[genes[1]]
        venny2 = venny2[venny2 >= threshold.value].index.to_list()
        venny2 = set(venny2)
        
        v1only0 = venny1.difference(venny2)  #new set with elements in venny1 but not in venny2
        v2only0 = venny2.difference(venny1)  #new set with elements in venny2 but not in venny1
        
        v1only = list(v1only0)
        v2only = list(v2only0)
        
        overlap_ab = [value for value in venny1 if value in venny2] 
        
        name1 = str(genes[0]) + " only"
        name2 = str(genes[1]) + " only"
        nameab = str(genes[0]) + " and " + str(genes[1])
        
        
        if len(genes) == 2:
            Title = (genes[0] + " / " + genes[1])
            excelTitle = (genes[0] + "_" + genes[1] + ".xlsx")

            if (len(venny1) >= 1) & (len(venny2) >= 1):
                la,lb,lc = len(v1only),len(v2only),len(overlap_ab)
                # now find the max
                max_len = max(la,lb,lc)
                if not max_len == la:
                    v1only.extend(['']*(max_len-la))
                if not max_len == lb:
                    v2only.extend(['']*(max_len-lb))
                if not max_len == lc:
                    overlap_ab.extend(['']*(max_len-lc))
                    
                venn_df = pd.DataFrame({name1:v1only,name2:v2only,nameab:overlap_ab}) 
                writer = pd.ExcelWriter(excelTitle, engine='openpyxl')
                venn_df.to_excel(writer, sheet_name='Sheet1')
                writer.save()
                display(venn_df)
                with out2:
                    clear_output()
                    plt.figure(figsize=(5, 5), facecolor="white")
                    v = venn2([set(venny1), set(venny2)], set_labels = (genes[0], genes[1]))
                    plt.title(Title)
                    plt.show()
                    savename = excelTitle + ".pdf"
                    plt.savefig(savename)

            else:
                with out2:
                    print("No Overlap: At least one gene is not expressed over a value of", threshold.value, "in any cell types:", 
                          genes[0], ": expressed in ", len(venny1), "cells; ", genes[1], ": expressed in ", len(venny2), "cells. ")

            
        elif len(genes) == 3:
            Title = (genes[0] + " / " + genes[1]+ " / " + genes[2])
            excelTitle = (genes[0] + "_" + genes[1] + "_" + genes[2] + ".xlsx")
            venny3 = mouse_means2.loc[genes[2]]
          

            venny3 = venny3[venny3 >= threshold.value].index.to_list()
            if (len(venny1) >= 1) & (len(venny2) >= 1) & (len(venny3) >= 1):
                name3 = str(genes[2]) + " only"
                namebc = str(genes[1]) + " and " + str(genes[2])
                nameac = str(genes[0]) + " and " + str(genes[2])
                nameabc = str(genes[0]) + " and " + str(genes[1])+ " and " + str(genes[2])
                
                venny30 = set(venny3)
                
                v3only = venny1.difference(venny2, venny30)
                v3only = list(v3only)
                
                overlap_bc = [value for value in venny2 if value in venny3]
                overlap_ac = [value for value in venny1 if value in venny3]
                overlap_abc = [value for value in venny3 if value in overlap_ab]

                la,lb,lc, ld, le, lf, lg  = len(v1only),len(v2only),len(v3only),len(overlap_ab),len(overlap_bc),len(overlap_ac),len(overlap_abc)
                # now find the max
                max_len = max(la,lb,lc,ld, le, lf, lg)
                if not max_len == la:
                    v1only.extend(['']*(max_len-la))
                if not max_len == lb:
                    v2only.extend(['']*(max_len-lb))
                if not max_len == lc:
                    v3only.extend(['']*(max_len-lc))
                if not max_len == ld:
                    overlap_ab.extend(['']*(max_len-ld))
                if not max_len == le:
                    overlap_bc.extend(['']*(max_len-le))
                if not max_len == lf:
                    overlap_ac.extend(['']*(max_len-lf))
                if not max_len == lg:
                    overlap_abc.extend(['']*(max_len-lg))

                venn_df = pd.DataFrame({name1:v1only,name2:v2only,name3:v3only,nameab:overlap_ab,namebc:overlap_bc,
                                                nameac:overlap_ac, nameabc: overlap_abc}) 
                writer = pd.ExcelWriter(excelTitle, engine='openpyxl')
                venn_df.to_excel(writer, sheet_name='Sheet1')
                writer.save() 
                display(venn_df)
                with out2:
                    clear_output()
                    plt.figure(figsize=(5, 5), facecolor="white")
                    v = venn3([set(venny1), set(venny2), set(venny3)], set_labels = (genes[0], genes[1], genes[2]))
                    plt.title(Title)
                    plt.show()
                    savename = excelTitle + ".pdf"
                    plt.savefig(savename)
 
            else:
                with out2:
                    print("No Overlap: At least one gene is not expressed over a value of", threshold.value, "in two of three cell types:", 
                          genes[0], ": expressed in ", len(venny1), "cells; ", genes[1], ": expressed in ", len(venny2), "cells; ",
                         genes[2], ": expressed in ", len(venny3), "cells.")

        

In [9]:
from ipywidgets import AppLayout
genesbox = widgets.VBox([threshold, genetitle,genes1, genes2, genes3, spacer, go_button])

out1 = widgets.Output(layout={'border': '1px solid black'})
out2 = widgets.Output(layout={'border': '1px solid black'})


box_layout = widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px')

with out2:
    genes1.value == 'Type Here'
    genes2.value == 'Type Here'
    genes3.value == 'Type Here'

#with out2:
#    clear_output()
    


In [10]:
AppLayout(header=welcome,
          left_sidebar=genesbox,
          center=out2,
          right_sidebar=None,
          footer=None,
          pane_widths=[2, 1, 1],
          pane_heights=[1, 4, '60px'],
         layout = box_layout)

AppLayout(children=(Label(value='Allen Transcriptomics Venn: Choose a threhold for expression and 2 or 3 genes…